## Using PostgreSQL with Jupyter

In [ ]:
!pip install psycopg2

### Minimal Verification

In [ ]:
import pandas as pd
import psycopg2 as pg2
import psycopg2.extras as pgex

In [ ]:
con = pg2.connect(host='this_postgres', user='postgres', database='postgres')
cur = con.cursor(cursor_factory=pgex.RealDictCursor)

In [ ]:
cur.execute("SELECT * FROM test;")

In [ ]:
pd.DataFrame(cur.fetchall())

In [ ]:
con.close()

### Loading Data into PostgreSQL

In [ ]:
con = pg2.connect(host='this_postgres', user='postgres', database='postgres')
cur = con.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute("""
BEGIN;
CREATE TABLE from_jupyter_test (
    _id INTEGER,
    name TEXT,
    list DOUBLE PRECISION[],
    vector BYTEA
);
COMMIT;
""")

In [ ]:
cur.execute("""
BEGIN;
INSERT INTO from_jupyter_test VALUES (1, 'spam', '{1,2,3,4,5}');
INSERT INTO from_jupyter_test VALUES (2, 'eggs', '{1,4,9,16,25}');
COMMIT;
""")

In [ ]:
cur.execute("""
SELECT * FROM from_jupyter_test;""")
pd.DataFrame(cur.fetchall())


In [ ]:
con.close()

### PostgreSQL Binary Type and Numpy

In [ ]:
cur.execute("""
SELECT list FROM from_jupyter_test;""")
results = cur.fetchall()
results

In [ ]:
import numpy as np
ary_1 = np.array(results[0]['list'])
ary_2 = np.array(results[1]['list'])

In [ ]:
bin_ary_1 = pg2.Binary(ary_1)
bin_ary_2 = pg2.Binary(ary_2)

In [ ]:
update_sql = """
BEGIN;
UPDATE from_jupyter_test
SET vector = {}
WHERE _id={};
COMMIT;
""".format(pg2.Binary(bin_ary_1), 1)
cur.execute(update_sql)

In [ ]:
update_sql = """
BEGIN;
UPDATE from_jupyter_test
SET vector = {}
WHERE _id={};
COMMIT;
""".format(pg2.Binary(bin_ary_2), 2)
cur.execute(update_sql)

In [ ]:
cur.execute("""
SELECT * FROM from_jupyter_test;""")
pd.DataFrame(cur.fetchall())

In [ ]:
cur.execute("""
SELECT vector FROM from_jupyter_test;""")
result = cur.fetchone()
result

In [ ]:
np.frombuffer(result['vector'])

In [ ]:
result = cur.fetchone()

In [ ]:
np.frombuffer(result['vector'])